<a href="https://colab.research.google.com/github/rtrochepy/astronomer/blob/main/day2_sp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Configuración
Primero, instala ChromaDB y el SDK de la API de Gemini en Python.


In [1]:
%pip install -U -q "google-generativeai>=0.8.3" chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 15.8 MB/s eta 0:00:

Es posible que veas un mensaje de ERROR relacionado con el resolvedor de dependencias de pip, pero esto no afecta el funcionamiento. Los paquetes están instalados y son compatibles para este laboratorio. No necesitas reiniciar el kernel.

In [2]:
import google.generativeai as genai
from IPython.display import Markdown


Configura tu clave API
Para ejecutar la siguiente celda, tu clave API debe estar almacenada en un secreto de Kaggle llamado GOOGLE_API_KEY.

Si aún no tienes una clave API, puedes obtenerla en AI Studio. Puedes encontrar instrucciones detalladas en la documentación.

Para que la clave esté disponible a través de los secretos de Kaggle, selecciona "Secrets" en el menú "Add-ons" y sigue las instrucciones para agregar tu clave o habilitarla para este notebook.


In [3]:
genai.configure(api_key='')

Explora los modelos disponibles
Usarás el método `embedContent` de la API para calcular embeddings en esta guía. Encuentra un modelo que lo soporte a través del endpoint `models.list`. También puedes encontrar más información sobre los modelos de embeddings en la página de modelos.

El modelo `text-embedding-004` es el modelo de embeddings más reciente, por lo que lo usarás para este ejercicio.


In [4]:
for m in genai.list_models():
    if "embedContent" in m.supported_generation_methods:
        print(m.name)


models/embedding-001
models/text-embedding-004


Datos
Aquí tienes un pequeño conjunto de documentos que usarás para crear una base de datos de embeddings.


In [5]:
DOCUMENTO1 = "Uso del sistema de control de clima. Tu Googlecar tiene un sistema de control de clima que te permite ajustar la temperatura y el flujo de aire en el auto. Para usarlo, utiliza los botones y perillas ubicados en la consola central.  Temperatura: La perilla de temperatura controla la temperatura dentro del auto. Gira la perilla en el sentido de las agujas del reloj para aumentar la temperatura o en sentido contrario para disminuirla. Flujo de aire: La perilla de flujo de aire controla la cantidad de aire que circula en el auto. Gira la perilla en el sentido de las agujas del reloj para aumentar el flujo de aire o en sentido contrario para disminuirlo. Velocidad del ventilador: La perilla de velocidad controla la velocidad del ventilador. Gira la perilla en el sentido de las agujas del reloj para aumentar la velocidad o en sentido contrario para disminuirla. Modo: El botón de modo te permite seleccionar el modo deseado. Los modos disponibles son: Automático: El auto ajustará automáticamente la temperatura y el flujo de aire para mantener un nivel cómodo. Frío: El auto soplará aire frío. Calor: El auto soplará aire caliente. Desempañar: El auto soplará aire caliente al parabrisas para desempañarlo."
DOCUMENTO2 = 'Tu Googlecar tiene una pantalla táctil grande que proporciona acceso a diversas funciones, incluyendo navegación, entretenimiento y control de clima. Para usar la pantalla táctil, simplemente toca el ícono deseado. Por ejemplo, puedes tocar el ícono de "Navegación" para obtener direcciones a tu destino o tocar el ícono de "Música" para reproducir tus canciones favoritas.'
DOCUMENTO3 = "Cambio de marchas. Tu Googlecar tiene una transmisión automática. Para cambiar de marcha, simplemente mueve la palanca de cambios a la posición deseada.  Estacionar: Esta posición se usa cuando estás estacionado. Las ruedas están bloqueadas y el auto no puede moverse. Reversa: Esta posición se usa para retroceder. Neutral: Esta posición se usa cuando estás detenido en un semáforo o en tráfico. El auto no está en marcha y no se moverá a menos que presiones el pedal del acelerador. Conducir: Esta posición se usa para avanzar. Baja: Esta posición se usa para conducir en nieve u otras condiciones resbaladizas."

documentos = [DOCUMENTO1, DOCUMENTO2, DOCUMENTO3]


Crear la base de datos de embeddings con ChromaDB
Crea una función personalizada para generar embeddings con la API de Gemini. En esta tarea, implementarás un sistema de recuperación, por lo que el `task_type` para generar embeddings de documentos será `retrieval_document`. Más adelante, usarás `retrieval_query` para los embeddings de consulta. Consulta la referencia de la API para ver la lista completa de tareas compatibles.


In [6]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry
class FuncionDeEmbeddingsGemini(EmbeddingFunction):
    # Especifica si se generan embeddings para documentos o consultas
    modo_documento = True
    def __call__(self, input: Documents) -> Embeddings:
        if self.modo_documento:
            tarea_embedding = "retrieval_document"
        else:
            tarea_embedding = "retrieval_query"
        politica_reintento = {"retry": retry.Retry(predicate=retry.if_transient_error)}
        respuesta = genai.embed_content(
            model="models/text-embedding-004",
            content=input,
            task_type=tarea_embedding,
            request_options=politica_reintento,
        )
        return respuesta["embedding"]


Ahora crea un cliente de base de datos Chroma que utilice la función personalizada de embeddings y llena la base de datos con los documentos definidos anteriormente.

In [7]:
%pip install chromadb

In [8]:
import chromadb

In [9]:
NOMBRE_BD = "googlecarbd"
funcion_embeddings = FuncionDeEmbeddingsGemini()
funcion_embeddings.modo_documento = True
cliente_chroma = chromadb.Client()
bd = cliente_chroma.get_or_create_collection(name=NOMBRE_BD, embedding_function=funcion_embeddings)
bd.add(documents=documentos, ids=[str(i) for i in range(len(documentos))])


Confirma que los datos se insertaron revisando la base de datos.

In [10]:
bd.count()
# También puedes echar un vistazo a los datos.
# bd.peek(1)


3

Recuperación: Encuentra documentos relevantes
Para buscar en la base de datos Chroma, utiliza el método `query`. Nota que también debes cambiar al modo `retrieval_query` para la generación de embeddings.


In [11]:
# Cambia al modo de consulta al generar embeddings.
funcion_embeddings.modo_documento = False

# Busca en la base de datos Chroma usando la consulta especificada.
consulta = "¿Cómo se usa la pantalla táctil para reproducir música?"

resultado = bd.query(query_texts=[consulta], n_results=1)
[[fragmento]] = resultado["documents"]

Markdown(fragmento)


Tu Googlecar tiene una pantalla táctil grande que proporciona acceso a diversas funciones, incluyendo navegación, entretenimiento y control de clima. Para usar la pantalla táctil, simplemente toca el ícono deseado. Por ejemplo, puedes tocar el ícono de "Navegación" para obtener direcciones a tu destino o tocar el ícono de "Música" para reproducir tus canciones favoritas.

Generación aumentada: Responde la pregunta
Ahora que has encontrado un fragmento relevante de los documentos (el paso de recuperación), puedes armar un prompt de generación para que la API de Gemini genere una respuesta final. Nota que en este ejemplo solo se recuperó un fragmento. En la práctica, especialmente cuando el tamaño de tus datos subyacentes es grande, querrás recuperar más de un resultado y dejar que el modelo Gemini determine qué fragmentos son relevantes para responder la pregunta.


In [12]:
fragmento_linea = fragmento.replace("\n", " ")
consulta_linea = consulta.replace("\n", " ")

# Este prompt es donde puedes especificar cualquier orientación sobre el tono, los temas que el modelo debe respetar o evitar.
prompt = f"""Eres un bot útil e informativo que responde preguntas utilizando texto del fragmento de referencia incluido a continuación.
Asegúrate de responder en una oración completa, siendo comprensivo, incluyendo toda la información de contexto relevante.
Sin embargo, estás hablando con una audiencia no técnica, así que asegúrate de descomponer conceptos complicados y
mantener un tono amigable y conversacional. Si el fragmento no es relevante para la respuesta, puedes ignorarlo.

PREGUNTA: {consulta_linea}
FRAGMENTO: {fragmento_linea}
"""
print(prompt)


Eres un bot útil e informativo que responde preguntas utilizando texto del fragmento de referencia incluido a continuación. 
Asegúrate de responder en una oración completa, siendo comprensivo, incluyendo toda la información de contexto relevante. 
Sin embargo, estás hablando con una audiencia no técnica, así que asegúrate de descomponer conceptos complicados y 
mantener un tono amigable y conversacional. Si el fragmento no es relevante para la respuesta, puedes ignorarlo.

PREGUNTA: ¿Cómo se usa la pantalla táctil para reproducir música?
FRAGMENTO: Tu Googlecar tiene una pantalla táctil grande que proporciona acceso a diversas funciones, incluyendo navegación, entretenimiento y control de clima. Para usar la pantalla táctil, simplemente toca el ícono deseado. Por ejemplo, puedes tocar el ícono de "Navegación" para obtener direcciones a tu destino o tocar el ícono de "Música" para reproducir tus canciones favoritas.



Ahora usa el método `generate_content` para generar una respuesta a la pregunta.

In [13]:
modelo = genai.GenerativeModel("gemini-1.5-flash-latest")
respuesta = modelo.generate_content(prompt)
Markdown(respuesta.text)


Para reproducir música en tu Googlecar, simplemente toca el ícono de "Música" en la gran pantalla táctil;  esto te permitirá acceder a tus canciones favoritas.


Próximos pasos
¡Felicidades por construir una aplicación de generación aumentada por recuperación!

Para aprender más sobre el uso de embeddings en la API de Gemini, consulta la Introducción a los embeddings o, para aprender fundamentos, estudia el capítulo de embeddings en el curso de Machine Learning Crash Course.

Para un sistema RAG alojado, consulta el servicio de Recuperación Semántica en la API de Gemini. Puedes implementar preguntas y respuestas sobre tus propios documentos en una sola solicitud o alojar una base de datos para obtener respuestas aún más rápidas.
